In [ ]:
!pip install python-dotenv

In [1]:
!pip install xmltodict

In [4]:
import os
import requests
import xmltodict
import pandas as pd
from dotenv import load_dotenv

# ===============================
# 1. 환경변수 로드
# ===============================
load_dotenv()
API_KEY = os.getenv("KIPRIS_API_KEY")

if not API_KEY:
    raise RuntimeError("KIPRIS_API_KEY가 설정되어 있지 않습니다.")

# ===============================
# 2. Endpoint
# ===============================
BASE_URL = (
    "http://plus.kipris.or.kr/"
    "kipo-api/kipi/patUtiModInfoSearchSevice/getAdvancedSearch"
)

# ===============================
# 3. 검색 조건
# ===============================
COMPANY_NAME = "터보파워텍"

params = {
    "serviceKey": API_KEY,

    # ⚠️ applicant는 실제로 무시되는 경우 많음
    "applicant": COMPANY_NAME,

    "patent": "true",
    "utility": "true",
    "numOfRows": 50,
    "pageNo": 1
}

# ===============================
# 4. API 요청
# ===============================
response = requests.get(BASE_URL, params=params, timeout=10)

# HTTP 자체 실패
if response.status_code != 200:
    raise RuntimeError(
        f"HTTP 오류 발생: {response.status_code}\n{response.text}"
    )

# ===============================
# 5. XML 파싱
# ===============================
try:
    data = xmltodict.parse(response.text)
except Exception:
    raise RuntimeError(
        "XML 파싱 실패. 응답이 XML이 아닙니다.\n"
        f"응답 원문:\n{response.text}"
    )

# ===============================
# 6. API 헤더 에러 검사 (🔥 핵심)
# ===============================
header = data.get("response", {}).get("header", {})

result_code = header.get("resultCode")
result_msg = header.get("resultMsg")

if result_code and result_code != "00":
    raise RuntimeError(
        f"KIPRIS API 오류\n"
        f"resultCode: {result_code}\n"
        f"resultMsg : {result_msg}"
    )

# ===============================
# 7. body / items 검사
# ===============================
body = data.get("response", {}).get("body")

if not body:
    raise RuntimeError(
        "API body가 없습니다. 검색 조건이 인식되지 않았습니다."
    )

items = body.get("items", {}).get("item")

if not items:
    raise RuntimeError(
        f"검색 결과 없음.\n"
        f"출원인명 '{COMPANY_NAME}'이(가) "
        f"현재 파라미터로는 인식되지 않았습니다.\n\n"
        f"⚠️ applicant 파라미터가 무시되었을 가능성이 큽니다."
    )

# ===============================
# 8. DataFrame 변환
# ===============================
if isinstance(items, dict):
    items = [items]

df = pd.DataFrame(items)

print("컬럼 목록:", df.columns.tolist())
print(df.head())
print(f"\n총 {len(df)}건")


RuntimeError: KIPRIS API 오류
resultCode: 10
resultMsg : INVALID REQUEST PARAMETER ERROR.